# In this notebook we will operate an array of LEDs from Python via artnet protocol

https://en.wikipedia.org/wiki/Art-Net

In [ ]:
#here is the only function we need to pack any values we want to send to a artnet byte packet and send it

import socket, struct

mysocket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 

def send_packet(data, sequence_counter=True, host="192.168.4.1"):
    universe_nr = 0
    
    sequence_counter = 255 if sequence_counter else 0
    packet = bytearray()
    packet.extend(map(ord, "Art-Net"))
    packet.append(0x00)          # Null terminate Art-Net
    packet.extend([0x00, 0x50])  # Opcode ArtDMX 0x5000 (Little endian)
    packet.extend([0x00, 0x0e])  # Protocol version 14
    
    if sequence_counter:
        sequence_counter += 1
        if sequence_counter > 255:
            sequence_counter = 1

    packet.append(sequence_counter)              # Sequence,
    packet.append(0x00)                                 # Physical
    packet.append(universe_nr & 0xFF)                   # Universe LowByte
    packet.append(universe_nr >> 8 & 0xFF)              # Universe HighByte

    packet.extend(struct.pack('>h', len(data)))  # Pack the number of channels Big endian
    packet.extend(data)
    mysocket.sendto(packet, (host, 0x1936))

## data is sequence of values 0-255 which are intensities of your lighting array. If you have 30 color LEDs, that means that you have to send maximum 30 x 3 = 90 values

Now make sure you're connected to the LEDs via WiFi and let's try it:

In [ ]:
values = [255, 0, 0]  # this should make the first LED to be red
send_packet(values)

Let's now make them all red but one by one with some sleep in time:

In [ ]:
debug = False

import time

values = []
for i in range(30):
    values.extend([255, 0, 0])
    if debug:
        print("we send: {}".format(values))
    send_packet(values)
    time.sleep(0.1)  # in sec

In [ ]:
# send 90 zeros to switch them off
send_packet([0] * 90)

We can make the colors all different:

In [ ]:
values = []
for i in range(30):
    red_value = int(255 / 30 * i)
    values.extend([red_value, 0, 255 - red_value])
    if debug:
        print("we send: {}".format(values))
    send_packet(values)
    time.sleep(0.1)  # in sec
send_packet([0] * 90)

We can use any desirable colormap, e.g. https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html (scroll to the bottom)

In [ ]:
import matplotlib.cm

colormap = matplotlib.cm.get_cmap('hsv')

print(colormap(0.0))
print(colormap(1.0))

def cm_to_rgb(value):
    assert 0.0 <= value <= 1.0
    return [int(255 * f) for f in colormap(value)[:3]]

print(cm_to_rgb(0.0))
print(cm_to_rgb(1.0))

In [ ]:
values = []
for i in range(30):
    values.extend(cm_to_rgb(i / 30.))
    if debug:
        print("we send: {}".format(values))
    send_packet(values)
    time.sleep(0.1)  # in sec
send_packet([0] * 90)

I want them to rotate!

In [ ]:
# we take the last values
while True:
    values = values[-3:] + values[:-3]
    for i in range(30):
        if debug:
            print("we send: {}".format(values))
        send_packet(values)
    time.sleep(0.05)  # in sec

Let's now make to move just one pixel

In [ ]:
for i in range(30):
    values = [0] * 90  # all zeros
    values[i * 3] = 255  # only i-th will be red
    if debug:
        print("we send: {}".format(values))
    send_packet(values)
    time.sleep(0.1)

We can speed up by reducing sleep time and make running infinite by using while loop

In [ ]:
sleep_time = 0.5
pixel_pos = 0
while True:
    values = [0] * 90
    values[pixel_pos * 3] = 255
    if debug:
        print("we send: {}".format(values))
    send_packet(values)
    time.sleep(sleep_time)
    # update values!!!
    pixel_pos += 1
    if pixel_pos == 30:
        pixel_pos = 0
    sleep_time = sleep_time * 0.99 # reduce sleep time

# _it's your turn to create_